In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import glob
import os

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.float_format ='{:,.3f}'.format

In [2]:
current_date = dt.date.today()
day = current_date.day - 1 # данные по продажам мы получаем за предыдущий день
month = current_date.month
year = current_date.year

In [3]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

df_dwh = pd.read_excel(latest_file)

In [9]:
print(latest_file)

\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\2024-05-20_2024-05-28.xlsx


In [10]:
# мы смотрим продажи со времени последнего мониторинга, для этого определяем день недели и в зависимости от этого получаем новые данные
data = dt.datetime.isoweekday(dt.date.today())
date_now = dt.datetime.now().date().strftime('%d.%m.%Y')
if data == 1:
    last_monitoring = (dt.date.today() - dt.timedelta(days=3)).strftime('%Y-%m-%d')
else:
    last_monitoring = (dt.date.today() - dt.timedelta(days=2)).strftime('%Y-%m-%d')
    
df_dwh = df_dwh[df_dwh['Дата'] >= f'{last_monitoring}']

In [11]:
df_dwh.sample(5)

,КБК,Дата,Продавец,Номер чека,ID продажи,Тов,Артикул поставщика,Количество,Номенклатура,Номер карты,Товарная группа,Товарное направление,Тип 1,Списание СберСпасибо,Группа наценки,Вес,Проба,Размер,ШК,Цена без скидки,Базовая скидка,Скидка в руб.,Название,Цена после скидки,Phone,Себестоимость,Сумма,Услуга,Общая сумма
206596,2-09-101,2024-05-28 07:26:00,Златова Наталья Николаевна,3618,Ch3-000778845,ТОВ1315638,2101143-00775,1,Товар,9000212237172,СИ СЕРЬГИ ИФ,СИ,СЕРЬГИ,14.880,KVI,3.078,925.000,0.000,"2,078,610,688,724.000","6,633.000",70.000,"1,651.700",СЕРЬГИ С ФИАН. РОД.,"1,651.700",+79102625473,979.450,"1,651.700",338.300,"1,990.000"
216635,5-02-081,2024-05-28 12:06:00,Мокеева Полина Викторовна,3696,Ch3-000779860,ТОВ0685368,ТОВ0685368,1,Сертификат,9000199772475,NaN,NaN,NaN,0.000,NaN,0.000,NaN,0.000,NaN,"1,000.000",0.000,"1,000.000",СЕРТИФИКАТ НОМ.1000,"1,000.000",+79272058183,NaN,"1,000.000",0.000,"1,000.000"
218536,6-01-107,2024-05-28 13:04:00,Бузулан Екатерина Александровна,2760,Ch4-000522872,ТОВ0887593,КО 15-00,1,Товар,9000120906313,БК КОЛЬЦА ОБРУЧ,БК,КОЛЬЦО ОБРУЧАЛЬНОЕ,0.000,ИФ.БК_ДОР,3.461,585.000,19.500,"2,078,606,368,484.000","52,548.000",25.000,"16,392.500",КОЛЬЦО ОБРУЧАЛЬНОЕ,"16,392.500",+79281710728,"10,669.540","16,392.500","3,357.500","19,750.000"
224067,1-01-228,2024-05-28 16:46:00,Малярова Любовь Борисовна,2601,Ch3-000781107,ТОВ1026168,СФ 555,1,Товар,5110004838747,ИФ СЕРЬГИ,ИФ,СЕРЬГИ,0.000,ИФ.БК_СР,2.489,585.000,0.000,"2,078,608,100,011.000","54,902.000",39.000,0.000,"СЕРЬГИ, КУБИЧЕСКИЙ Ц","15,023.000",9517280451,"9,421.820","15,023.000","3,077.000","18,100.000"
205718,4-04-039,2024-05-28 06:58:00,Ваганова София Андреевна,3457,Ch2-000829250,ТОВ1241610,200-1046KR-PR,1,Товар,9000212234270,СИ СЕРЬГИ ЦВ/К,СИ,СЕРЬГИ,0.000,СИ_СЕРЬГИ_СРЕДН,2.774,925.000,0.000,"2,078,610,188,458.000","4,648.000",40.000,"1,693.530",СЕРЬГИ С ЮВ.КРИСТ.ПО,"1,303.100",+79175249671,858.680,"1,303.100",266.900,"1,570.000"


In [12]:
df_dwh['Товарная группа'].str.contains('ЦБ|БК').value_counts()

Товарная группа
False    10861
True      4400
Name: count, dtype: int64

In [13]:
df_dwh = df_dwh[(df_dwh['Товарное направление'].isin(['БК', 'ЦБ'])) & (df_dwh['Проба'] == 585.0)]

In [14]:
df_dwh.head()

,КБК,Дата,Продавец,Номер чека,ID продажи,Тов,Артикул поставщика,Количество,Номенклатура,Номер карты,Товарная группа,Товарное направление,Тип 1,Списание СберСпасибо,Группа наценки,Вес,Проба,Размер,ШК,Цена без скидки,Базовая скидка,Скидка в руб.,Название,Цена после скидки,Phone,Себестоимость,Сумма,Услуга,Общая сумма
202412,3-09-049,2024-05-28 00:56:00,Вербицкая Лилия Игоревна,3890,Ch2-000828412,ТОВ01260428,311-00-0035-30555,1,Товар,9000147277779,ЦБ ЦЕПИ,ЦБ,ЦЕПЬ,0.000,"ЦБ_ПУСТОТЕЛЫЕ <=1,5 ГРАММ",0.952,585.000,40.000,"2,078,608,358,307.000","22,368.000",31.000,"6,432.500",ЦЕПЬ ЯКОРЬ ПУСТ,"6,432.500",+79243376273,"3,584.290","6,432.500","1,317.500","7,750.000"
202418,3-09-019,2024-05-28 01:00:00,Тарских Ксения Александровна,3581,Ch2-000828411,ТОВ01262538,431-01-0080-37654,1,Товар,7651032893660,ЦБ БРАСЛЕТЫ,ЦБ,БРАСЛЕТ,138.490,ЦБ_ПУСТОТЕЛЫЕ >3 ГРАММ,4.688,585.000,22.000,"2,078,607,527,398.000","79,296.000",6.000,"30,959.000",БРАСЛЕТ БИСМАРК ПУСТ,"30,959.000",+79148070595,"16,394.580","30,959.000","6,341.000","37,300.000"
202422,3-09-049,2024-05-28 01:02:00,Вербицкая Лилия Игоревна,429,Ch2-000828399,ТОВ0881058,14.43_15,1,Товар,9000207895233,БК СЕРЬГИ,БК,СЕРЬГИ-КОНГО,0.000,ИФ.БК_ДЕШ,1.061,585.000,1.500,"2,078,607,768,244.000","23,067.000",34.000,"6,349.500",СЕРЬГИ-КОНГО,"6,349.500",+79025291038,"3,740.480","6,349.500","1,300.500","7,650.000"
202444,3-09-048,2024-05-28 01:38:00,Томашик Ирина Михайловна,3243,Ch2-000828416,ТОВ1307111,41-46050-11,1,Товар,9000212217686,ЦБ БРАСЛЕТЫ,ЦБ,БРАСЛЕТ,241.210,ЦБ_РУЧНОЕ ПЛЕТЕНИЕ >3 ГРАММ,3.808,585.000,18.000,"2,078,607,515,534.000","65,352.000",11.000,"24,153.000",БРАСЛЕТ БИСМАРК,"24,153.000",(924)441-28-53,"17,388.300","24,153.000","4,947.000","29,100.000"
202445,3-09-048,2024-05-28 01:38:00,Томашик Ирина Михайловна,3243,Ch2-000828416,ТОВ1134866,"НБ12-087ПГ D0,5",1,Товар,9000212217686,ЦБ БРАСЛЕТЫ,ЦБ,БРАСЛЕТ,67.550,"ЦБ_ПУСТОТЕЛЫЕ <=1,5 ГРАММ",1.050,585.000,16.000,"2,078,608,381,507.000","23,533.000",31.000,"6,764.500",БРАСЛЕТ ЛАВ ПУСТ,"6,764.500",(924)441-28-53,"4,605.040","6,764.500","1,385.500","8,150.000"


In [15]:
df_dwh['Товарное направление'].unique()

array(['ЦБ', 'БК'], dtype=object)

In [16]:
df_dwh['Название'].isna().sum()

0

In [17]:
df_dwh['Название'] = df_dwh['Название'].fillna('золотое изделие')

In [18]:
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('каучук')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('шнурок')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('текстиль')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('хлопок')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('шелк')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('на нитке')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('кожа')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('жемчуг')))]

In [19]:
df_dwh['Дата'] = pd.to_datetime(df_dwh['Дата'], format='%Y-%m-%d')
df_dwh['Дата'] = pd.to_datetime(df_dwh['Дата'].apply(lambda x: x.strftime("%Y-%m-%d")))

In [20]:
filtr_date = df_dwh['Дата'].max()

In [21]:
df_dwh['Дата'].min()

Timestamp('2024-05-28 00:00:00')

In [22]:
df_dwh = df_dwh.query("`Дата` <= @filtr_date")

In [23]:
df_dwh['Дата'].unique()

<DatetimeArray>
['2024-05-28 00:00:00']
Length: 1, dtype: datetime64[ns]

In [24]:
df_dwh['Товарное направление'].value_counts()

Товарное направление
БК    1552
ЦБ    1261
Name: count, dtype: int64

In [25]:
df_dwh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2813 entries, 202412 to 224975
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   КБК                   2813 non-null   object        
 1   Дата                  2813 non-null   datetime64[ns]
 2   Продавец              2813 non-null   object        
 3   Номер чека            2813 non-null   int64         
 4   ID продажи            2813 non-null   object        
 5   Тов                   2813 non-null   object        
 6   Артикул поставщика    2813 non-null   object        
 7   Количество            2813 non-null   int64         
 8   Номенклатура          2813 non-null   object        
 9   Номер карты           2711 non-null   object        
 10  Товарная группа       2813 non-null   object        
 11  Товарное направление  2813 non-null   object        
 12  Тип 1                 2813 non-null   object        
 13  Списание СберСпа

In [30]:
tn = df_dwh.groupby(by = 'Товарная группа', as_index= False)[['Себестоимость', 'Вес', 'Общая сумма']].agg('sum')

In [31]:
tn['Наценка продажи оффлайн'] =  tn['Общая сумма'] / tn['Себестоимость'] - 1

In [32]:
tn = tn[~tn['Товарная группа'].str.contains('ОПТ БК|БК ПЕЧАТКИ|ОПТ КОНГО|ОПТ ОБРУЧИ')]

In [33]:
tn['price_per_gramm'] = ((tn['Общая сумма'] / tn['Вес']).round(0))

In [34]:
tn['price_per_gramm'] = tn['price_per_gramm'].replace(',', '')
tn['price_per_gramm'] = tn['price_per_gramm'].astype('int')

In [35]:
tn

,Товарная группа,Себестоимость,Вес,Общая сумма,Наценка продажи оффлайн,price_per_gramm
0,БК КОЛЬЦА,"979,925.560",334.214,"2,201,170.000",1.246,6586
1,БК КОЛЬЦА ОБРУЧ,"8,595,048.140","2,955.989","18,045,055.270",1.099,6105
3,БК ПОДВЕС ДЕКОР,"617,353.550",197.075,"1,389,288.000",1.250,7050
4,БК ПОДВЕС КУЛЬТ,"2,875,881.650",940.755,"6,490,609.040",1.257,6899
5,БК СЕРЬГИ,"2,396,601.810",785.581,"5,392,223.750",1.250,6864
8,ЦБ БРАСЛЕТЫ,"9,491,666.860","2,854.096","18,771,628.580",0.978,6577
9,ЦБ ЦЕПИ,"18,603,445.260","5,768.846","38,966,465.830",1.095,6755


In [36]:
print('Средняя цена продажи оффлайн: ', (tn['Общая сумма'].sum() / tn['Вес'].sum()).round(0))

Средняя цена продажи оффлайн:  6595.0


In [38]:
tn.columns = ['tg', 'cost', 'weight', 'price', 'Наценка продажи оффлайн', 'price_per_gramm']

In [39]:
tn.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sales_offline\offline_sales_{date_now}.xlsx', index = False)

In [40]:
tn = tn.drop(columns=['cost', 'weight', 'price', 'Наценка продажи оффлайн'])
tn.columns = ['tg', '585*Золотой оффлайн продажи']

In [33]:
pd.read_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx').merge(tn, how = 'left', on = 'tg').to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx', index=False)